# Testing script

Script for testing out classes created for Multinational Retail Data Centralisation project. 

**Note** Update docstrings for classes and methods accordingly as functionality develops. 


## Load in modules of classes

In [ ]:
import database_utils
import data_extraction
import data_cleaning
import pandas as pd
import numpy as np

In [ ]:
connector = database_utils.DatabaseConnector()
extractor = data_extraction.DataExtractor()
cleaning = data_cleaning.DataCleaning()

Utilise class methods to connect to AWS RDS database and retrieve list of table names from postgres database.

In [ ]:
creds = connector.read_db_creds()
engine = connector.init_db_engine()
db_list = connector.list_db_tables()

Extract table data into a panda dataframe. 

In [ ]:
rds_df = extractor.read_rds_table('legacy_users', engine)
rds_df.head()

## Cleaning user data testing

This is testing scripts for cleaning up phone_numbers. 

### Phone number cleaning

This is a bit more complicated than expected

In [ ]:
rds_df['phone_number'] = np.where(rds_df['phone_number'].str.startswith('A-Z|a-z'), np.nan, rds_df['phone_number'])

isd_code_map = { "GB": "+44", "DE": "+49", "US": "+1" }

def correct_phone_number(row):
  import re
  # Remove special chars other than digits, `+` and letters used for extension e.g. `x`, `ext` (following keeps all alphabets).
  result = re.sub("[^A-Za-z\d\+]", "", row["phone_number"])
  
  # Prefix ISD code by matching country code.
  if not result.startswith(isd_code_map[row["country_code"]]):
    result = isd_code_map[row["country_code"]] + result

  # Remove `0` that follows ISD code.
  if result.startswith(isd_code_map[row["country_code"]] + "0"):
    result = result.replace(isd_code_map[row["country_code"]] + "0", isd_code_map[row["country_code"]])
  return result

rds_df["Corrected Phone Number"] = rds_df.apply(correct_phone_number, axis=1)


In [ ]:
#regex_expression = '^(?:(?:\(?(?:0(?:0|11)\)?[\s-]?\(?|\+)44\)?[\s-]?(?:\(?0\)?[\s-]?)?)|(?:\(?0))(?:(?:\d{5}\)?[\s-]?\d{4,5})|(?:\d{4}\)?[\s-]?(?:\d{5}|\d{3}[\s-]?\d{3}))|(?:\d{3}\)?[\s-]?\d{3}[\s-]?\d{3,4})|(?:\d{2}\)?[\s-]?\d{4}[\s-]?\d{4}))(?:[\s-]?(?:x|ext\.?|\#)\d{3,4})?$' #Our regular expression to match
#rds_df['phone_number'] = np.where(rds_df['country'] == 'United Kingdom', rds_df['phone_number'].str.match(regex_expression), np.nan)

np.where(rds_df['phone_number'].str.startswith('A-Z|a-z'), np.nan, rds_df['phone_number'])

Cleaning up of User data using pandas. 
These test methods will be finalised and entered into the data_cleaning class methods. 

Check address formatting issues - seems there could be issues with /n - regrex



In [ ]:
clean_rds_df = cleaning.clean_user_data(rds_df)

print(type(clean_rds_df))
print(clean_rds_df.to_string())

Upload cleaned up data to sales_database

In [ ]:
connector.upload_to_db(clean_rds_df, 'dim_users')

## Card Details data

In [ ]:
#extracting data
card_df = extractor.retrieve_pdf_data()

In [ ]:
#cleaning card details data
cleaning = data_cleaning.DataCleaning()
clean_card_df = cleaning.clean_card_data(card_df)
clean_card_df

In [ ]:
#Upload cleaned data to local database
connector.upload_to_db(clean_card_df, 'dim_card_details')

# Store details

endpoint = 'https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/number_stores'

header = {'x-api-key':'yFBQbwXe9J3sd6zWVAMrK6lcxxr0q1lr2PT6DDMX'}

In [ ]:
num_stores = extractor.list_number_of_stores('https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/number_stores',{'x-api-key':'yFBQbwXe9J3sd6zWVAMrK6lcxxr0q1lr2PT6DDMX'} )

In [ ]:
stores_df2 = extractor.retrieve_stores_data(num_stores,'https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/store_details/')

In [ ]:
stores_df = stores_df2.copy()

In [ ]:
#testing cleaning methods
clean_stores_df = cleaning.called_clean_store_data(stores_df)
clean_stores_df.head(10)

In [ ]:
#Upload cleaned data to local database
connector.upload_to_db(clean_stores_df, 'dim_store_details')

## Product details

Milestone 2 - Task 6

In [ ]:
product_df = extractor.extract_from_s3('s3://data-handling-public/products.csv')

In [ ]:
product_df.info()

In [ ]:
product_df_weight = cleaning.convert_product_weights(product_df)

In [ ]:
clean_product_df = cleaning.clean_products_data(product_df_weight)

In [ ]:
connector.upload_to_db(clean_product_df, 'dim_products')

## Order data

Create a method in DataCleaning called clean_orders_data which will clean the orders table data.

You should remove the columns, first_name, last_name and 1 to have the table in the correct form before uploading to the database.

You will see that the orders data contains column headers which are the same in other tables.

This table will act as the source of truth for your sales data and will be at the center of your star based database schema.



Once cleaned upload using the upload_to_db method and store in a table called orders_table,

In [ ]:
creds = connector.read_db_creds()
engine = connector.init_db_engine()
db_list = connector.list_db_tables()

In [4]:
orders_df = extractor.read_rds_table('orders_table', engine)

In [ ]:
orders_df.head()

In [5]:
clean_orders_df = cleaning.clean_orders_data(orders_df)

#note had to delete level_0 column as was causing duplicate col error, which couldn't find resolution
clean_orders_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120123 entries, 0 to 120122
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   index             120123 non-null  int64 
 1   date_uuid         120123 non-null  object
 2   user_uuid         120123 non-null  object
 3   card_number       120123 non-null  int64 
 4   store_code        120123 non-null  object
 5   product_code      120123 non-null  object
 6   product_quantity  120123 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 6.4+ MB


In [6]:
connector.upload_to_db(clean_orders_df, 'orders_table')